In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder \
    .appName("fact_category_product") \
    .config("spark.cores.max", "1") \
    .config("spark.executor.memory", "2g") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "hive") \
    .config("spark.sql.catalog.iceberg.uri", "thrift://hive-metastore:9083") \
    .config("spark.sql.catalog.iceberg.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.catalog.iceberg.warehouse", "s3a://warehouse/") \
    .config("spark.sql.catalog.iceberg.s3.endpoint", "http://minio:9000") \
    .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/23 18:42:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_product = spark.read \
    .format('iceberg') \
    .load('iceberg.silver.dm_product') \
    .select(
        'product_id',
        'category_id',
        'name',
        'price',
        'original_price',
        'discount',
        'discount_rate',
        'quantity_sold',
        'stock_qty',
        'created_date'
    )

df_category = spark.read \
    .format('iceberg') \
    .load('iceberg.silver.dm_category') \
    .select(
        'category_id',
        'category_name'
    )

In [4]:
df = df_product.join(df_category, df_product['category_id'] == df_category['category_id'], 'inner') \
    .drop(df_category['category_id'])
df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- price: long (nullable = true)
 |-- original_price: long (nullable = true)
 |-- discount: long (nullable = true)
 |-- discount_rate: float (nullable = true)
 |-- quantity_sold: long (nullable = true)
 |-- stock_qty: long (nullable = true)
 |-- created_date: date (nullable = true)
 |-- category_name: string (nullable = true)



In [5]:
group_column = ['category_id', 'category_name']

df = df.groupBy(*group_column).agg(
    F.countDistinct('product_id').alias('total_product'),
    F.coalesce(F.sum(F.coalesce(col('quantity_sold'), F.lit(0))), F.lit(0)).alias('total_quantity_sold'),
    F.coalesce(F.sum(F.coalesce(col('price'), F.lit(0)) * F.coalesce(col('quantity_sold'), F.lit(0))), F.lit(0)).alias('revenue'),
    F.round(F.coalesce(F.avg(col('price')), F.lit(0)), 2).alias('avg_price'),
    F.round(
        F.when(
            F.countDistinct('product_id') == 0,
            F.lit(0)
        ).otherwise(
            F.coalesce(F.sum(F.coalesce(col('quantity_sold'), F.lit(0))), F.lit(0)) / F.countDistinct('product_id')
        ),
        2
    ).alias('avg_units_sold_per_product')
)

df.printSchema()

root
 |-- category_id: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- total_product: long (nullable = false)
 |-- total_quantity_sold: long (nullable = false)
 |-- revenue: long (nullable = false)
 |-- avg_price: double (nullable = true)
 |-- avg_units_sold_per_product: double (nullable = true)



In [6]:
df.show()

+-----------+--------------------+-------------+-------------------+----------+-------------+--------------------------+
|category_id|       category_name|total_product|total_quantity_sold|   revenue|    avg_price|avg_units_sold_per_product|
+-----------+--------------------+-------------+-------------------+----------+-------------+--------------------------+
|      68231|     Máy đo nhịp tim|            1|                 99|  59301000|     599000.0|                      99.0|
|      27170|         Máy xịt rửa|           15|                 68|  92426250|   2245306.53|                      4.53|
|       4342|     Dụng cụ mài dao|           11|                 72|   8030900|    175147.18|                      6.55|
|      27196|Thiết bị đo lường...|           18|                 56|  36912386|   1782451.33|                      3.11|
|      23260|   Miếng lót bồn cầu|            9|                237|   8937000|     312000.0|                     26.33|
|        159|  Drinks & Beverage

In [7]:
df = df.withColumn("ngay_cap_nhat", current_timestamp())

df.write \
    .format('iceberg') \
    .mode('overwrite') \
    .saveAsTable('iceberg.gold.fact_category_product')

In [8]:
spark.stop()